DOES COLAB

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder


In [4]:
titanic = pd.read_csv("~/data/titanic_dataset.csv")

In [5]:
titanic.head

<bound method NDFrame.head of      Class     Sex    Age Survived
0      3rd    Male  Child       No
1      3rd    Male  Child       No
2      3rd    Male  Child       No
3      3rd    Male  Child       No
4      3rd    Male  Child       No
...    ...     ...    ...      ...
2196  Crew  Female  Adult      Yes
2197  Crew  Female  Adult      Yes
2198  Crew  Female  Adult      Yes
2199  Crew  Female  Adult      Yes
2200  Crew  Female  Adult      Yes

[2201 rows x 4 columns]>

In [6]:
#Titanic column names to lower case:
titanic.columns = titanic.columns.str.lower()
titanic.columns

#Count missing values in each column:
titanic.isnull().sum()

class       0
sex         0
age         0
survived    0
dtype: int64

In [10]:
#convert to numeric values:
le = LabelEncoder()

titanic["sex"] = le.fit_transform(titanic["sex"])
titanic["class"] = le.fit_transform(titanic["class"])
titanic["survived"] = le.fit_transform(titanic["survived"])
titanic["age"] = le.fit_transform(titanic["age"])


In [11]:
#Split titanic into test train validation:
titanic_train = titanic.sample(frac=0.8, random_state=0)
titanic_test = titanic.drop(titanic_train.index)
titanic_train, titanic_val = titanic_train.sample(frac=0.8, random_state=0), titanic_train.drop(titanic_train.index)

#Split into features and labels:
titanic_train_features = titanic_train.copy()
titanic_train_labels = titanic_train_features.pop('survived')
titanic_val_features = titanic_val.copy()
titanic_val_labels = titanic_val_features.pop('survived')
titanic_test_features = titanic_test.copy()
titanic_test_labels = titanic_test_features.pop('survived')

len(titanic_train_features), len(titanic_train_labels)

(1409, 1409)

In [12]:
#Instantiate a deep sequential model:
model = keras.Sequential([
    keras.layers.Dense(512, activation= "relu", input_shape=(1409, 3)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation= "relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation= "relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-09-17 09:17:17.846795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-17 09:17:17.846913: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1409, 512)         2048      
_________________________________________________________________
dropout (Dropout)            (None, 1409, 512)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 1409, 256)         131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1409, 256)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1409, 64)          16448     
_________________________________________________________________
dense_3 (Dense)              (None, 1409, 1)           65        
Total params: 149,889
Trainable params: 149,889
Non-trainable params: 0
__________________________________________________

In [14]:
#Compile the model:
model.compile(optimizer= "adam",
                loss='binary_crossentropy',
                metrics=['accuracy'])


In [15]:
#Send the model to GPU and train
with tf.device('/gpu:0'):
    model.fit(titanic_train_features, titanic_train_labels, epochs=50, batch_size=64, validation_data=(titanic_val_features, titanic_val_labels))


Epoch 1/50


2022-09-17 09:17:38.772666: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-17 09:17:38.772907: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-17 09:17:38.934443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 5s 76ms/step - loss: 0.5760 - accuracy: 0.7097
Epoch 2/50
23/23 [==============================] - 0s 8ms/step - loss: 0.5411 - accuracy: 0.7700
Epoch 3/50
23/23 [==============================] - 0s 8ms/step - loss: 0.5281 - accuracy: 0.7800
Epoch 4/50
23/23 [==============================] - 0s 8ms/step - loss: 0.5019 - accuracy: 0.7842
Epoch 5/50
23/23 [==============================] - 0s 9ms/step - loss: 0.4913 - accuracy: 0.7878
Epoch 6/50
23/23 [==============================] - 0s 8ms/step - loss: 0.4926 - accuracy: 0.7835
Epoch 7/50
23/23 [==============================] - 0s 8ms/step - loss: 0.4840 - accuracy: 0.7786
Epoch 8/50
23/23 [==============================] - 0s 8ms/step - loss: 0.4861 - accuracy: 0.7828
Epoch 9/50
23/23 [==============================] - 0s 8ms/step - loss: 0.5228 - accuracy: 0.7757
Epoch 10/50
23/23 [==============================] - 0s 8ms/step - loss: 0.4840 - accuracy: 0.7857
Epoch 11/50
23/23 [==========

In [16]:
#Evaluate the model:
model.evaluate(titanic_test_features, titanic_test_labels)


 1/14 [=>............................] - ETA: 2s - loss: 0.2659 - accuracy: 1.0000

2022-09-17 09:17:56.452800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - 0s 10ms/step - loss: 0.5055 - accuracy: 0.7750


[0.5054545402526855, 0.7750000357627869]